In [ ]:
import joblib
import cartopy
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt
from pathlib import Path
from collections import Counter
import geoviews as gv
import holoviews as hv
# Mis funciones
from data_utils import load_axa_data
from data_utils import wgs84_to_mercator

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%matplotlib inline
hv.extension('bokeh')

In [ ]:
df_axa = pd.read_parquet('../data/processed/accidentes_axa.parquet', engine='pyarrow')
print(df_axa.shape)
df_axa.head()

In [ ]:
coords_hora = [(wgs84_to_mercator(lat, lon), hora) for lat, lon, hora in zip(df_axa.latitud, df_axa.longitud, df_axa.hora)]

coords_hora = [{'y': y, 'x': x, 'hora': hora} for (y, x), hora in coords_hora]
df_coords = pd.DataFrame(coords_hora)
df_coords.head(3)

In [ ]:
# Holoviews dataset
coords_ds = hv.Dataset(df_coords, kdims=['x', 'y'], vdims=['hora'])
coords_ds.data.head()

In [ ]:
# Mapa
width = 1200
height = 700

tiles_opts = dict(width=width, height=height, bgcolor='black', xaxis=None, yaxis=None, show_grid=False)
tiles = gv.tile_sources.Wikipedia.clone(crs=cartopy.crs.GOOGLE_MERCATOR).options(**tiles_opts) 
tiles

In [ ]:
hex_two_distributions = hv.HexTiles(coords_ds)

hex_two_distributions = hex_two_distributions.options(
    width=width, height=height,
    alpha=0.5, cmap='viridis', logz=False,
    tools=['hover'], colorbar=True, min_count=1,
)
hex_two_distributions

In [ ]:
mapa = tiles * hex_two_distributions
mapa

In [ ]:
# save html
renderer = hv.renderer('bokeh')
renderer.save(mapa, '../figures/mapa_accidentes')

In [ ]:
# Por hora 

width = 900
height = 700

def generate_maps(hora):
    # hextiles, se filtran por hora
    coords_ds_por_hora = coords_ds.select(hora=hora)
    # print(coords_ds_por_hora.data.shape)
    hextiles = hv.HexTiles(coords_ds_por_hora)
    hextiles = hextiles.options(
        width=width, height=height,
        alpha=0.5, cmap='viridis', logz=False,
        tools=['hover'], colorbar=True, min_count=1,
    )
    # Map tiles
    tiles_opts = dict(width=width, height=height, bgcolor='black', 
                      xaxis=None, yaxis=None, show_grid=False)
    tiles_hora = gv.tile_sources.Wikipedia.clone(
        crs=cartopy.crs.GOOGLE_MERCATOR).options(**tiles_opts) 
    mapa_hora = tiles_hora * hextiles
    return mapa_hora


horas = [str(i).zfill(2) for i in range(0, 24)]
dmap = hv.DynamicMap(generate_maps, kdims='hora')
dmap.redim.values(hora=horas)


In [ ]:
# Load all axa
# path_axa = Path('../data/raw/')

# dfs_axa = joblib.Parallel(n_jobs=-1, verbose=0)(
#     joblib.delayed(load_axa_data)(path.as_posix())
#     for path in path_axa.iterdir() 
#     if path.as_posix().endswith('.geojson')
# )

# df_axa = pd.concat(dfs_axa, axis=0, ignore_index=True, sort=True)
# print(df_axa.shape)
# df_axa.drop('codigo_pos_c_254', axis=1).to_parquet('../data/processed/accidentes_axa.parquet', engine='pyarrow')
# df_axa.head()